In [2]:
import pandas as pd
import numpy as np
import os, json
import spacy
nlp = spacy.load('en_core_web_sm')

## Read rssData 

In [3]:
path_to_json = 'rssData/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
?print(json_files)

Object `print(json_files)` not found.


In [4]:
data_list = []
for root, dirs, files in os.walk('rssData/'):
     for file in files:
        with open(os.path.join(root, file), "r") as auto:
            json_data = auto.read()
            #print(json_data)
            data_list.append(json_data)

In [5]:
from ast import literal_eval
tup = literal_eval(data_list[0])

header_title = []
text = []

for i in range(len(data_list)):
    tup = literal_eval(data_list[i])
    header_title.append(tup['header_title'])
    text.append(tup['text'])

df = pd.DataFrame()
df['header_title'] = header_title
df['text'] = text

## Cleaning

In [6]:
# Remove Empty Text Values

df['text'].replace('', np.nan, inplace=True)
df.dropna(subset=['text'], inplace=True)
df = df.reset_index()

# Remove text if lenght is less than 200
df = df.loc[df.text.str.len() > 200].reset_index(drop=True)

# function to clean data

#lower case
#remove stop words
#lemmatization

def cleanData(doc):
    doc = doc.lower()
    doc = nlp(doc)
    tokens = [tokens.lower_ for tokens in doc]
    tokens = [tokens for tokens in doc if (tokens.is_stop == False)]
    tokens = [tokens for tokens in tokens if (tokens.is_punct == False)]
    final_token = [token.lemma_ for token in tokens]
    
    return " ".join(final_token)

In [7]:
df['clean'] = df.apply(lambda row:cleanData (row['text']),axis=1)

## Implement NLP techniques

https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.Y-Zq4XbMI2w

### TF-IDF Model for doc # i

In [8]:
### Calculate TF-IDF for doc_number #i

i=500
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer() 
cv=CountVectorizer() 
# this steps generates word counts for the words in all docs 
word_count_vector=cv.fit_transform(df['clean'])
# Compute idf weights
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
all_text_df_tfidf = tfidf_transformer.fit_transform(word_count_vector)

### Calculate TF-IDF

# count matrix 
count_vector=cv.transform(df['clean']) 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

feature_names = cv.get_feature_names_out() 
#get tfidf vector for first document 
N_document_vector=tf_idf_vector[i] 
#print the scores 
df_TFIDF = pd.DataFrame(N_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df_TFIDF=df_TFIDF.sort_values(by=["tfidf"],ascending=False)

<font color=green>the more unique a word is to our first document, the higher the score

### POS Tagging

In [9]:
def POS(clean):
    clean = nlp(clean)
    for token in clean:
        return(token.pos_) 

# Word Extraction Based on TF-IDF for Words Longer than K and POS_tag

In [10]:
# Title
print('Title is:',df['header_title'][i])
print('\n')

# Most frequent words
df_TFIDF = df_TFIDF.reset_index(names=['word', 'tfidf'])
df_TFIDF['pos'] = df_TFIDF.apply(lambda row:POS (row['word']),axis=1)

# Words lenght longer than k
k=3
df_TFIDF_1 = df_TFIDF[(df_TFIDF['word'].str.len()) > k]

# POS_part : NOUN, VERB

POS_tag = 'VERB'
print('Most frequent words based on', POS_tag)
df_TFIDF_1[df_TFIDF_1['pos']==POS_tag].head(10)[['word','tfidf']].reset_index(drop=True)

#10 Most frequent words
# print('Most Frequent Words Based on TF-IDF:')
# df_TFIDF_1.head(10)

Title is: Securing Cloud is Everyone’s Responsibility


Most frequent words based on VERB


,word,tfidf
0,shared,0.130733
1,provider,0.122412
2,understand,0.073463
3,rhymes,0.069484
4,twain,0.065367
5,securing,0.062445
6,everyoneâ,0.058327
7,reinvent,0.058327
8,hear,0.057575
9,implement,0.057210


## TF-IDF for POS_tag

In [40]:
print('Most frequent words based on NOUN and VERB')
df_TFIDF_1[(df_TFIDF_1['pos'] == 'NOUN') | (df_TFIDF_1['pos'] == 'VERB')].head(10)[['word','tfidf']]

Most frequent words based on NOUN and VERB


,word,tfidf
3,turla,0.180677
6,certify,0.163476
8,website,0.139387
9,security,0.135657
10,protection,0.135477
11,attack,0.135477
12,researcher,0.134107
13,group,0.134107
14,ukraine,0.125853
16,bitdefender,0.125853
